In [1]:
from main import DataImporter

data_importer = DataImporter()
data_importer.import_data()

ds_train = data_importer.get_train_data()
ds_test = data_importer.get_test_data()
ds_validation = data_importer.get_validation_data()

display(ds_train.head())
display(ds_test.head())
display(ds_validation.head())

ModuleNotFoundError: No module named 'datasets'